# Poisson Surface Reconstruction

Let $X = \lbrace x_1, \ldots, x_n \rbrace \subset \mathbb R^3$ be points sampled at the boundary $\partial M$ of a solid $M$ and denote by $f\colon \partial M \to \mathbb R^3$ the inward unit normal vector function. 

Let $u$ be the indicator function of $M$. Then, pictorically one would expect
$$
     \nabla u(x) = f(x) \qquad x\in\partial M.
$$
However, the gradient of the indicator function is unbounded at the boundary of $M$. Therefore, we consider the regularized indicator $u_{\epsilon} = u\ast \eta_{\epsilon}$ where $\eta_{\epsilon}$ is a $C^{\infty}$ function supported on $\partial_{\epsilon} M = \partial M + B_{\epsilon}$. Denote this set of functions as $C^{\infty}_{\epsilon}$. 

Notice that $u_{\epsilon}$ is an smooth function in $\mathbb R^3$.

For the regularized problem we relax the objective as well. Instead of point-wise equality, we ask for least-squares minimization:
$$
    \min_{v} \int_{\partial_{\epsilon} M} \lVert \nabla v(x) - f(x) \rVert^2 dx
    =
    \min_{v} J(v)
    \qquad
    v\in C^{\infty}_{\epsilon}
$$
Notice, however, that this formulation is ill-posed since $f$ is not defined outside of $\partial M$. Therefore, we first extend $f$ to the fattest set $\partial_{\epsilon} M$. The extension is based on the following lemma.

**Lemma** 
$$
    \nabla u_{\epsilon}(x) = \int_{\partial M} \eta_{\epsilon}(x-y) f(y) \, dy 
$$

<div style="color: blue"><strong>Proof</strong>

It's a consequence of the divergence theorem (applied by rows)
$$
\begin{align}
    \nabla u_{\epsilon}(x)
    &= \nabla \int_{M} \eta_{\epsilon}(x-y) \, dy
    \\&= \int_{M} \nabla \eta_{\epsilon}(x-y) \, dy
    \\&= \int_{M} (\eta_{\epsilon}(x-y) \mathbf I) \cdot \nabla \, dy
    \\&= \int_{\partial M} (\eta_{\epsilon}(x-y) \mathbf I) \cdot f(y) \, dy
    \\&= \int_{\partial M} \eta_{\epsilon}(x-y) f(y) \, dy.
\end{align}
$$

</div>

Thus, we extend $f$ as $\nabla u_{\epsilon}$ outside of $\partial M$. Now the formulation is properly defined.

Since $J$ is a convex and positive function, it has a unique minimizer. The first order conditions,
$$
\begin{align}
    0 
    &= J(v)[w]
    \\&= \frac{d}{dt}\vert_{t=0} \int \lVert \nabla \left( v(x) + t w(x) \right) - f(x) \rVert^2 dx
    \\&= \frac{d}{dt}\vert_{t=0} \int \sum_{j=1}^{3} (\partial_j (v(x) + tw(x)) - f_j(x))^2 \, dx
    \\&= \int 2 \sum_{j=1}^{3} (\partial_j v(x) - f_j(x)) \partial_j w(x) \, dx
    \\&= 2 \int (\nabla v(x) - f(x)) \cdot \nabla w(x) \, dx
\end{align}
$$
yields that
$$
    \int_{\partial_{\epsilon} M} \nabla v(x)\cdot \nabla w(x) = \int_{\partial_{\epsilon} M} f(x) \nabla w(x)
    \qquad \forall w \in C^{\infty}_{\epsilon}
    \tag{VF}
$$
which is the weak formulation of the Poisson problem. Using integration by parts in the right hand side yields:
$$
    -\Delta v(x) = \nabla \cdot f(x) \qquad x\in\partial_{\epsilon} M.
$$
Therein the name of the method.

## Solving the Least-Square problems

We will apply the finite element method (FEM) to solve (VF). FEM works by discretizing the function space by using "shape functions" $\psi_1, \ldots, \psi_n$ such that every $w$ and $v$ can be written as a linear combination of the shape functions $\psi_j$. i.e.
$$
    w(x) = a_1(w) \psi_1(x) + \ldots + a_n(w) \psi_n(x).
$$
shapes functions should be taken to be linearly independent.
The weak formulation (VF), then, can be simplified to
\begin{align*}
    \int \nabla v\cdot \nabla w &= \int f\cdot \nabla w\\
    \implies
    \int \nabla v \cdot \nabla\psi_j &= \int f\cdot \nabla\psi_j \qquad j=1,\ldots, n\\
    \implies
    \sum_{i=1}^{n} a_i(v) \int \nabla\psi_i \cdot \nabla\psi_j &= \int f\cdot \nabla\psi_j \qquad j=1,\ldots,n.
\end{align*}
That is, we are left with the linear system $\mathbf A \vec a = \vec f$ where $\mathbf A_{ji} = \int \nabla\psi_i\cdot \nabla\psi_j$,
$\vec a_i = a_i(v)$ and $\vec f_j = \int f\cdot \nabla\psi_j$.

Take $\mathcal O$ an adaptative octree of depth $D$ such that every sample point $x_j$ lives in a leaf node at depth $D$. Choose a function $F$ and define the shape functions at every leaf node $o \in \mathcal O$ as
$$
    F_o(q) = \frac{1}{o.w^3}\, F\left( \frac{q-o.c}{o.w} \right)
$$
where $o.c$ is the center and $o.w$ the width of the node. Define $V$ as the space generated by functions $F_o$.

The challenge is to make the choice of $F$ good enough as to be able to represent enough functions in the sense that $V\subset H^1_0(\partial_{\epsilon} M)$ and that the induced linear system is invertible and sparse.

## Actually Solving the Least-Squares problem

The algoritm works as follows:

1. Choose a regularizer $\eta_{\epsilon}$.
2. Extend $f$.
3. Implement an adaptative tree-like structure.
4. Choose a bump function $F$ and assemble the formulation.
5. Solve the linear system.

### 2D Problem

In [1]:
import numpy as np

class QuadTree:

    def __init__(X):
        pass

# References

[1] Kazhdan, Michael, Matthew Bolitho, and Hugues Hoppe. "Poisson surface reconstruction." 2006. Available at: https://hhoppe.com/proj/poissonrecon/